<a href="https://colab.research.google.com/github/dukavinod/Machine-Learning/blob/main/Telecomchurn_spark_logregression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [283]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [284]:
!wget -q https://dlcdn.apache.org/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz

In [285]:
!tar xf spark-3.2.2-bin-hadoop3.2.tgz

In [286]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/content/spark-3.2.2-bin-hadoop3.2'

In [287]:
# Install library for finding Spark
!pip install -q findspark
# Import the libary
import findspark
# Initiate findspark
findspark.init()
# Check the location for Spark
findspark.find()

'/content/spark-3.2.2-bin-hadoop3.2'

In [288]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

In [289]:
# Import a Spark function from library
from pyspark.sql.functions import *

In [290]:
churn = spark.read.csv('churn_data.csv',header = True)

In [291]:
customer = spark.read.csv('customer_data.csv',header = True)

In [292]:
internet = spark.read.csv('internet_data.csv',header = True)

In [293]:
churn_customer = churn.join(customer,on = 'customerID',how = 'inner')

In [294]:
telecom = churn_customer.join(internet,on = 'customerID',how = 'inner')

In [295]:
# all the data merged into a single dataframe called telecom.
telecom.columns 

['customerID',
 'tenure',
 'PhoneService',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies']

In [296]:
# finding the number of items in each category of value.
for x in telecom.columns:
  telecom.select(x).groupby(x).count().show()

+----------+-----+
|customerID|count|
+----------+-----+
|3668-QPYBK|    1|
|6234-RAAPL|    1|
|1894-IGFSG|    1|
|6982-SSHFK|    1|
|5859-HZYLF|    1|
|6479-OAUSD|    1|
|2592-YKDIF|    1|
|6718-BDGHG|    1|
|3195-TQDZX|    1|
|4248-QPAVC|    1|
|5668-MEISB|    1|
|5802-ADBRC|    1|
|2712-SYWAY|    1|
|2011-TRQYE|    1|
|7244-KXYZN|    1|
|0953-LGOVU|    1|
|3623-FQBOX|    1|
|3692-JHONH|    1|
|3528-HFRIQ|    1|
|7661-CPURM|    1|
+----------+-----+
only showing top 20 rows

+------+-----+
|tenure|count|
+------+-----+
|     7|  131|
|    51|   68|
|    15|   99|
|    54|   68|
|    11|   99|
|    69|   95|
|    29|   72|
|    42|   65|
|    64|   80|
|     3|  200|
|    30|   72|
|    34|   65|
|    59|   60|
|     8|  123|
|    22|   90|
|    28|   57|
|    16|   80|
|    52|   80|
|    35|   88|
|     0|   11|
+------+-----+
only showing top 20 rows

+------------+-----+
|PhoneService|count|
+------------+-----+
|          No|  682|
|         Yes| 6361|
+------------+-----+

+----

In [297]:
telecomdf = telecom.select('*',expr("case when PhoneService = 'Yes' then '1' else '0' end").alias('phoneService_alias')
,expr("case when gender = 'Male' then '1' else '0' end").alias('gender_alias')
,expr("case when Partner = 'Yes' then '1' else '0' end").alias('partner_alias')
,expr("case when Dependents = 'Yes' then '1' else '0' end").alias('dependent_alias')
,expr("case when PaperlessBilling = 'Yes' then '1' else '0' end").alias('paperlessBilling_alias')
,expr("case when Churn = 'Yes' then '1' else '0' end").alias('churn_alias'))




In [298]:
telecomdf.show()

+----------+------+------------+--------------+----------------+--------------------+--------------+------------+-----+------+-------------+-------+----------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------+-------------+---------------+----------------------+-----------+
|customerID|tenure|PhoneService|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|gender|SeniorCitizen|Partner|Dependents|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|phoneService_alias|gender_alias|partner_alias|dependent_alias|paperlessBilling_alias|churn_alias|
+----------+------+------------+--------------+----------------+--------------------+--------------+------------+-----+------+-------------+-------+----------+----------------+----

In [299]:
telecom_new = telecomdf.drop('PhoneService','PaperlessBilling','gender','Partner','Dependents','Churn')


In [300]:
telecom_new.show(5)

+----------+------+--------------+--------------------+--------------+------------+-------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+------------------+------------+-------------+---------------+----------------------+-----------+
|customerID|tenure|      Contract|       PaymentMethod|MonthlyCharges|TotalCharges|SeniorCitizen|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|phoneService_alias|gender_alias|partner_alias|dependent_alias|paperlessBilling_alias|churn_alias|
+----------+------+--------------+--------------------+--------------+------------+-------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+------------------+------------+-------------+---------------+----------------------+-----------+
|7590-VHVEG|     1|Month-to-month|    Electronic check|      

In [301]:
# import necessary SQL Datatypes, functions and Window operations for the transformations required
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [302]:
cat_cols = ['Contract','PaymentMethod','InternetService']

In [303]:
from pyspark.ml.feature import OneHotEncoder , StringIndexer , VectorAssembler

In [304]:
from pyspark.ml import Pipeline

In [305]:
telecom_new.show(5)

+----------+------+--------------+--------------------+--------------+------------+-------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+------------------+------------+-------------+---------------+----------------------+-----------+
|customerID|tenure|      Contract|       PaymentMethod|MonthlyCharges|TotalCharges|SeniorCitizen|   MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|phoneService_alias|gender_alias|partner_alias|dependent_alias|paperlessBilling_alias|churn_alias|
+----------+------+--------------+--------------------+--------------+------------+-------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+------------------+------------+-------------+---------------+----------------------+-----------+
|7590-VHVEG|     1|Month-to-month|    Electronic check|      

In [306]:
cat_cols = ['Contract','PaymentMethod','SeniorCitizen','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']

In [307]:
for item in cat_cols:
  telecom_new.select(item).groupby(item).count().show()

+--------------+-----+
|      Contract|count|
+--------------+-----+
|Month-to-month| 3875|
|      One year| 1473|
|      Two year| 1695|
+--------------+-----+

+--------------------+-----+
|       PaymentMethod|count|
+--------------------+-----+
|Credit card (auto...| 1522|
|        Mailed check| 1612|
|Bank transfer (au...| 1544|
|    Electronic check| 2365|
+--------------------+-----+

+-------------+-----+
|SeniorCitizen|count|
+-------------+-----+
|            0| 5901|
|            1| 1142|
+-------------+-----+

+----------------+-----+
|   MultipleLines|count|
+----------------+-----+
|No phone service|  682|
|              No| 3390|
|             Yes| 2971|
+----------------+-----+

+---------------+-----+
|InternetService|count|
+---------------+-----+
|    Fiber optic| 3096|
|             No| 1526|
|            DSL| 2421|
+---------------+-----+

+-------------------+-----+
|     OnlineSecurity|count|
+-------------------+-----+
|                 No| 3498|
|              

In [308]:
telecom_new.show()

+----------+------+--------------+--------------------+--------------+------------+-------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------+-------------+---------------+----------------------+-----------+
|customerID|tenure|      Contract|       PaymentMethod|MonthlyCharges|TotalCharges|SeniorCitizen|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|phoneService_alias|gender_alias|partner_alias|dependent_alias|paperlessBilling_alias|churn_alias|
+----------+------+--------------+--------------------+--------------+------------+-------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------+-------------+--------

In [309]:
for x in cat_cols:
  indexer = StringIndexer(inputCol = x,outputCol = x+'_indexed' )
  telecom_new = indexer.fit(telecom_new).transform(telecom_new)
  
  #pipeline = Pipeline(stages = stages)
  #pipelineModel = pipeline.fit(telecomdf)
  #telecomdf = pipelineModel.transform(telecomdf)






In [310]:
telecom_new.show()

+----------+------+--------------+--------------------+--------------+------------+-------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------+-------------+---------------+----------------------+-----------+----------------+---------------------+---------------------+---------------------+-----------------------+----------------------+--------------------+------------------------+-------------------+-------------------+-----------------------+
|customerID|tenure|      Contract|       PaymentMethod|MonthlyCharges|TotalCharges|SeniorCitizen|   MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|phoneService_alias|gender_alias|partner_alias|dependent_alias|paperlessBilling_alias|churn_alias|Contract_indexed|PaymentMethod_indexed|SeniorCitizen_indexed|Mult

In [311]:
ind_col = ['Contract_indexed','PaymentMethod_indexed','SeniorCitizen_indexed','MultipleLines_indexed','InternetService_indexed','OnlineSecurity_indexed','OnlineBackup_indexed','DeviceProtection_indexed','TechSupport_indexed','StreamingTV_indexed','StreamingMovies_indexed']

In [312]:
for y in ind_col:
  encoder = OneHotEncoder(inputCol = y, outputCol = y+'vector')
  telecom_new = encoder.fit(telecom_new).transform(telecom_new)

In [313]:
telecom_new.show(20)

+----------+------+--------------+--------------------+--------------+------------+-------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------+-------------+---------------+----------------------+-----------+----------------+---------------------+---------------------+---------------------+-----------------------+----------------------+--------------------+------------------------+-------------------+-------------------+-----------------------+----------------------+---------------------------+---------------------------+---------------------------+-----------------------------+----------------------------+--------------------------+------------------------------+-------------------------+-------------------------+-----------------------------+
|customerID|tenure|      Contract|       PaymentMethod|MonthlyCharges|TotalCharges|SeniorCitizen|   Multip

In [314]:
telecom_new.columns

['customerID',
 'tenure',
 'Contract',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'SeniorCitizen',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'phoneService_alias',
 'gender_alias',
 'partner_alias',
 'dependent_alias',
 'paperlessBilling_alias',
 'churn_alias',
 'Contract_indexed',
 'PaymentMethod_indexed',
 'SeniorCitizen_indexed',
 'MultipleLines_indexed',
 'InternetService_indexed',
 'OnlineSecurity_indexed',
 'OnlineBackup_indexed',
 'DeviceProtection_indexed',
 'TechSupport_indexed',
 'StreamingTV_indexed',
 'StreamingMovies_indexed',
 'Contract_indexedvector',
 'PaymentMethod_indexedvector',
 'SeniorCitizen_indexedvector',
 'MultipleLines_indexedvector',
 'InternetService_indexedvector',
 'OnlineSecurity_indexedvector',
 'OnlineBackup_indexedvector',
 'DeviceProtection_indexedvector',
 'TechSupport_indexedvector',
 'StreamingTV_indexedvector',
 'StreamingMovies_ind

In [315]:
telecom_new.show(5)

+----------+------+--------------+--------------------+--------------+------------+-------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+------------------+------------+-------------+---------------+----------------------+-----------+----------------+---------------------+---------------------+---------------------+-----------------------+----------------------+--------------------+------------------------+-------------------+-------------------+-----------------------+----------------------+---------------------------+---------------------------+---------------------------+-----------------------------+----------------------------+--------------------------+------------------------------+-------------------------+-------------------------+-----------------------------+
|customerID|tenure|      Contract|       PaymentMethod|MonthlyCharges|TotalCharges|SeniorCitizen|   MultipleLines|InternetService|OnlineSecur

In [316]:
feat_col = ['tenure',
 'MonthlyCharges',
 'TotalCharges',
 'phoneService_alias',
 'gender_alias',
 'partner_alias',
 'dependent_alias',
 'paperlessBilling_alias',
 'Contract_indexedvector',
 'PaymentMethod_indexedvector',
 'SeniorCitizen_indexedvector',
 'MultipleLines_indexedvector',
 'InternetService_indexedvector',
 'OnlineSecurity_indexedvector',
 'OnlineBackup_indexedvector',
 'DeviceProtection_indexedvector',
 'TechSupport_indexedvector',
 'StreamingTV_indexedvector',
 'StreamingMovies_indexedvector']

In [317]:
telecom_new = telecom_new.withColumn('MonthlyCharges', telecom_new["MonthlyCharges"].cast(DecimalType(10,5)))
telecom_new = telecom_new.withColumn('TotalCharges', telecom_new["TotalCharges"].cast(DecimalType(10,5)))
telecom_new = telecom_new.withColumn('tenure', telecom_new["tenure"].cast('int'))
telecom_new = telecom_new.withColumn('phoneService_alias', telecom_new["phoneService_alias"].cast(BooleanType()))
telecom_new = telecom_new.withColumn('gender_alias', telecom_new["gender_alias"].cast(BooleanType()))
telecom_new = telecom_new.withColumn('partner_alias', telecom_new["partner_alias"].cast(BooleanType()))
telecom_new = telecom_new.withColumn('dependent_alias', telecom_new["dependent_alias"].cast(BooleanType()))
telecom_new = telecom_new.withColumn('paperlessBilling_alias', telecom_new["paperlessBilling_alias"].cast(BooleanType()))
telecom_new = telecom_new.withColumn('churn_alias', telecom_new["churn_alias"].cast(BooleanType()))

In [318]:
assembler = VectorAssembler(inputCols = feat_col ,outputCol = 'features')
telecom_new = assembler.transform(telecom_new)

In [319]:
telecom_new.show()

+----------+------+--------------+--------------------+--------------+------------+-------------+----------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------+-------------+---------------+----------------------+-----------+----------------+---------------------+---------------------+---------------------+-----------------------+----------------------+--------------------+------------------------+-------------------+-------------------+-----------------------+----------------------+---------------------------+---------------------------+---------------------------+-----------------------------+----------------------------+--------------------------+------------------------------+-------------------------+-------------------------+-----------------------------+--------------------+
|customerID|tenure|      Contract|       PaymentMethod|MonthlyCharges|TotalCharges|Se

In [320]:
telecom_new.columns

['customerID',
 'tenure',
 'Contract',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'SeniorCitizen',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'phoneService_alias',
 'gender_alias',
 'partner_alias',
 'dependent_alias',
 'paperlessBilling_alias',
 'churn_alias',
 'Contract_indexed',
 'PaymentMethod_indexed',
 'SeniorCitizen_indexed',
 'MultipleLines_indexed',
 'InternetService_indexed',
 'OnlineSecurity_indexed',
 'OnlineBackup_indexed',
 'DeviceProtection_indexed',
 'TechSupport_indexed',
 'StreamingTV_indexed',
 'StreamingMovies_indexed',
 'Contract_indexedvector',
 'PaymentMethod_indexedvector',
 'SeniorCitizen_indexedvector',
 'MultipleLines_indexedvector',
 'InternetService_indexedvector',
 'OnlineSecurity_indexedvector',
 'OnlineBackup_indexedvector',
 'DeviceProtection_indexedvector',
 'TechSupport_indexedvector',
 'StreamingTV_indexedvector',
 'StreamingMovies_ind

In [321]:
telechurn = telecom_new

In [322]:
telechurn = telechurn.drop('customerID',
 'tenure',
 'Contract',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'SeniorCitizen',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'phoneService_alias',
 'gender_alias',
 'partner_alias',
 'dependent_alias',
 'paperlessBilling_alias',
 'Contract_indexed',
 'PaymentMethod_indexed',
 'SeniorCitizen_indexed',
 'MultipleLines_indexed',
 'InternetService_indexed',
 'OnlineSecurity_indexed',
 'OnlineBackup_indexed',
 'DeviceProtection_indexed',
 'TechSupport_indexed',
 'StreamingTV_indexed',
 'StreamingMovies_indexed',
 'Contract_indexedvector',
 'PaymentMethod_indexedvector',
 'SeniorCitizen_indexedvector',
 'MultipleLines_indexedvector',
 'InternetService_indexedvector',
 'OnlineSecurity_indexedvector',
 'OnlineBackup_indexedvector',
 'DeviceProtection_indexedvector',
 'TechSupport_indexedvector',
 'StreamingTV_indexedvector',
 'StreamingMovies_indexedvector')

In [323]:
telechurn.show()

+-----------+--------------------+
|churn_alias|            features|
+-----------+--------------------+
|      false|(30,[0,1,2,5,7,8,...|
|      false|(30,[0,1,2,3,4,11...|
|       true|(30,[0,1,2,3,4,7,...|
|      false|(30,[0,1,2,4,12,1...|
|       true|(30,[0,1,2,3,7,8,...|
|       true|(30,[0,1,2,3,7,8,...|
|      false|(30,[0,1,2,3,4,6,...|
|      false|(30,[0,1,2,8,11,1...|
|       true|(30,[0,1,2,3,5,7,...|
|      false|(30,[0,1,2,3,4,6,...|
|      false|[13.0,49.95,587.4...|
|      false|(30,[0,1,2,3,4,9,...|
|      false|(30,[0,1,2,3,4,5,...|
|       true|(30,[0,1,2,3,4,7,...|
|      false|(30,[0,1,2,3,4,7,...|
|      false|(30,[0,1,2,3,5,6,...|
|      false|(30,[0,1,2,3,11,1...|
|      false|(30,[0,1,2,3,4,6,...|
|       true|(30,[0,1,2,3,5,6,...|
|      false|(30,[0,1,2,3,7,8,...|
+-----------+--------------------+
only showing top 20 rows

